# Lab 12: Grid Localization using Bayes Filter (Real Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

2022-05-18 15:46:40,855 | INFO     |: Logger demo_notebook.log initialized.


In [2]:
# Start the plotter
START_PLOTTER()

# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [3]:
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

        # Data collection stuff
        self.dist_data = []
        self.angle_data = []

        self.setup_notify()
        

    def setup_notify(self):
        print("Notifications started")
        ble.start_notify(ble.uuid['RX_TOF1'], self.tof1_callback_handler)
        print(ble.uuid['RX_TOF1'])
        ble.start_notify(ble.uuid['RX_IMU_PITCH'], self.imu_pitch_callback_handler)

    def tof1_callback_handler(self, uuid, byte_array):
        new_tof = self.ble.bytearray_to_float(byte_array)
        # print(f"Got TOF data: {new_tof}")
        self.dist_data.append(new_tof)

    def imu_pitch_callback_handler(self, uuid, byte_array):
        new_angle = self.ble.bytearray_to_float(byte_array)
        # print(f"Got angle data: {new_angle}")
        self.angle_data.append( new_angle )
        
    def set_params(self, pid_p, pid_min_power, imu_hz):
        self.ble.send_command(CMD.SET_PID, f"{pid_p}|{pid_min_power}|{imu_hz}"); 

    def set_motor_calib(self, calib):
        self.ble.send_command(CMD.SET_MOTOR_CALIB, calib)
        
    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        return (0,0,0)
        # raise NotImplementedError("get_pose is not implemented")

    def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        self.ble.send_command(CMD.OBSERVE, None)
        time.sleep(15)
        self.ble.send_command(CMD.GET_DATA, None)
        while len(self.dist_data) < 18 or len(self.angle_data) < 18:
            time.sleep(2)
            if len(self.dist_data) < 2 or len(self.angle_data) < 2:
                self.dist_data = []
                self.angle_data = []
                self.ble.send_command(CMD.GET_DATA, None)
            pass
        # Convert to meters, and account for how the sensor isn't
        # mounted in the middle of the robot
        self.dist_data = [x / 1000 for x in self.dist_data]
        temp_angles = []
        temp_sum = 0
        for i,angle in enumerate(self.angle_data):
            temp_sum -= angle
            temp_angles.append(temp_sum)
        self.angle_data = temp_angles
        print(f"TOF: {self.dist_data}")
        print(f"Angles: {self.angle_data}")
        return np.array(self.dist_data).reshape((18,1)), np.array(self.angle_data).reshape((18,1))


In [4]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2022-05-18 15:46:41,635 | INFO     |: Looking for Artemis Nano Peripheral Device: C0:07:21:8D:B3:44
2022-05-18 15:46:44,902 | INFO     |: Connected to C0:07:21:8D:B3:44


In [5]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

Notifications started
531f9dab-8ff2-4eb6-9489-c402953d27b6
2022-05-18 15:46:45,983 | INFO     |:  | Number of observations per grid cell: 18
2022-05-18 15:46:45,983 | INFO     |:  | Precaching Views...


/Users/main/Documents/Classes/2022-sp/ECE-4960/ece4960-fast-robots-code/lab10/ECE4960-sim-release-release-mirror/localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2022-05-18 15:46:50,020 | INFO     |:  | Precaching Time: 4.035 secs
2022-05-18 15:46:50,021 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-18 15:46:50,022 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run an update step of the Bayes Filter

In [6]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# def set_params(self, pid_p, pid_min_power, imu_hz):
robot.dist_data = []
robot.angle_data = []
robot.set_params(1.3, 100, 52)
robot.set_motor_calib(1.3)
    
# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot Odom and GT
current_odom, current_gt = robot.get_pose()
cmdr.plot_gt(current_gt[0], current_gt[1])
cmdr.plot_odom(current_odom[0], current_odom[1])

2022-05-18 15:46:50,246 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-18 15:46:50,247 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
TOF: [1.1546666259765626, 1.2443333740234375, 0.982, 1.041, 0.867, 0.632, 0.5493333129882813, 0.5743333129882813, 0.5426666870117187, 0.46733334350585937, 0.43, 0.4803333435058594, 0.6316666870117188, 0.7956666870117187, 0.7726666870117187, 1.2683333740234375, 2.480333251953125, 1.2596666259765625]
Angles: [19.661479949951172, 39.61729431152344, 60.10760498046875, 80.17953109741211, 100.30665969848633, 120.02510070800781, 140.3813819885254, 160.7784767150879, 181.12991905212402, 201.30490493774414, 221.01189422607422, 240.84733390808105, 260.5692958831787, 280.4806308746338, 300.717565536499, 320.6394691467285, 340.6297798156738, 360.61553955078125]
2022-05-18 15:47:07,402 | INFO     |: Update Step
2022-05-18 15:47:07,406 | INFO     |:      | Update Time: 0.003 secs
2022-05-18 15:47:07,407 | INFO     |: Bel

ValueError: too many values to unpack (expected 2)

In [ ]:
import matplotlib.pyplot as plt
tof = [0.364, 0.5603333129882813, 0.4983333435058594, 0.403, 0.5966666870117188, 2.016, 0.716, 0.4966666564941406, 1.844, 2.116333251953125, 2.376, 0.224, 0.2496666717529297, 0.32433334350585935, 0.389, 0.487, 0.408, 0.38833334350585935]
angle = [19.711673736572266, 39.30105209350586, 59.348920822143555, 79.80359649658203, 99.47217178344727, 119.46056365966797, 139.58079147338867, 159.5949478149414, 179.2961254119873, 199.308687210083, 218.94672775268555, 239.1281852722168, 259.3975887298584, 279.3796195983887, 299.0354709625244, 318.8495616912842, 338.4791736602783, 358.6300964355469]
cartesan_x = []
cartesan_y = []
for r,theta in zip(tof,angle):
    cartesan_x.append(r * np.cos(np.deg2rad(theta)))
    cartesan_y.append(r * np.sin(np.deg2rad(theta)))
plt.scatter(cartesan_x, cartesan_y)